<a href="https://colab.research.google.com/github/kelsdoerksen/giga-connectivity/blob/main/GeoCLIP_Extraction_For_Connectivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## C02 - Use GeoCLIP embeddings

Simple example of how to obtain pretrained GeoCLIP embeddings. Read the paper here:[https://arxiv.org/abs/2309.16020](https://arxiv.org/abs/2309.16020). First install the geoclip package (see [https://github.com/VicenteVivan/geo-clip](https://github.com/VicenteVivan/geo-clip))

In [1]:
!pip install geoclip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 5.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [2]:
from geoclip import LocationEncoder
import torch
import torch.nn as nn
import pandas as pd
import ast

Load the pretrained model directly.

In [3]:
model = LocationEncoder()

Obtain GeoCLIP location embeddings.

In [4]:
# Get [lon, lat] of schools as float.64 tensor to extract embeddings for

def get_coords(df):
  """
  Function to return coords of school locations
  as 2D tensor to extract GeoCLIP embeddings for
  in order lon, lat
  """

  total_coords = []
  for i in range(len(df)):
    coord = torch.tensor((df.loc[i]['lon'], df.loc[i]['lat']))
    total_coords.append(coord)

  locations = torch.stack(total_coords)

  return locations

In [5]:
# Processing data for locations for the embeddings to be extracted from
RWA_df = pd.read_csv('RWA_id_info.csv')

In [6]:
# Get coordinates for aoi of interest
coords = get_coords(RWA_df)

In [7]:
model.eval()
with torch.no_grad():
  x = model(coords.flip(1).float()).detach().cpu()

In [8]:
identifying_info_df = RWA_df[['giga_id_school', 'connectivity', 'lat', 'lon', 'split', 'fid']]
emb_df = pd.DataFrame(x.numpy())

In [9]:
emb_df_labelled = pd.concat([identifying_info_df, emb_df], axis=1)

In [10]:
# Split into Train/Test/Val
emb_train = emb_df_labelled[emb_df_labelled['split'] =='Train']
emb_test = emb_df_labelled[emb_df_labelled['split'] =='Test']
emb_val = emb_df_labelled[emb_df_labelled['split'] =='Val']

In [11]:
# Export to dataframe
emb_train.to_csv('RWA_GeoCLIP_embeddings_TrainingData.csv')
emb_test.to_csv('RWA_GeoCLIP_embeddings_TestingData.csv')
emb_val.to_csv('RWA_GeoCLIP_embeddings_ValData.csv')